In [ ]:
import pickle5 as pickle
with open('../../data/COVID-QA/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [ ]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]

In [ ]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = False
if filtering == True:
    filtering_threshold = 0.5
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #We're consider scibert because pubmedbert assigns very high similarity for both related/unrelated terms.
    checkpoint = 'allenai/scibert_scivocab_uncased'
    
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        '''
        we're taking less than here since the similarity scores for related terms seem to be lower than 
        unrelated ones.
        '''
        if similarity.item() < filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

In [ ]:
import pandas as pd
pd.DataFrame(context_dict.items(), columns = ['ent', 'text']).to_parquet('wiki_corpus.parquet', index=False)

In [ ]:
from pymed import PubMed

# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = ['covid-19', 'hiv-1']

# Execute the query against the API
results = pubmed.query(query, max_results=500)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    abstract = article.abstract

    # Show information about the article
    print(
        f'{abstract}\n'
    )

In [10]:
import json
with open('../../data/RadQA/radqa-a-question-answering-dataset-to-'
          'improve-comprehension-of-radiology-reports-1.0.0/train.json', 'r') as f:
    s = json.load(f)

In [13]:
for article in s["data"]:
    for paragraph in article["paragraphs"]:
        context = paragraph["context"].strip()
        document_id = paragraph["document_id"]
        for qa in paragraph["qas"]:
            question = qa["question"].strip()
            is_impossible = qa["is_impossible"]
            id_ = qa["id"]

            answer_starts = [answer["answer_start"] for answer in qa["answers"]]
            answers = [answer["text"].strip() for answer in qa["answers"]]

            # Features currently used are "context", "question", and "answers".
            # Others are extracted here for the ease of future expansions.
            print( id_, {
                "document_id": document_id,
                "context": context,
                "question": question,
                "is_impossible": is_impossible,
                "id": id_,
                "answers": {
                    "answer_start": answer_starts,
                    "text": answers,
                },
            })
            break
        break

796653_2_1_I {'document_id': '796653_I', 'context': 'IMPRESSION:  Subdural hematomas with blood products of different ages.\n Question vescular abnormality in left suprasellar space.  Findings were\n discussed with Dr. [**Last Name (STitle) 8620**] at 9:25 am on [**2191-8-5**].  An MRI of the brain and MRA\n of the COW is recommended.', 'question': 'Is there any significant change in bleeding?', 'is_impossible': True, 'id': '796653_2_1_I', 'answers': {'answer_start': [], 'text': []}}
789660_1_2_I {'document_id': '789660_I', 'context': 'CONCLUSION:\n 1)  Small bowel obstruction at the area of ileal anastomosis, likely related\n to fecal obstruction and impaction.\n\n 2)  There is contrast material within the kidneys; this may represent ATN or\n continued renal excretion of orally administered contrast.  It is recommended\n that a creatinine level be rechecked.', 'question': 'Does the small bowel look obstructed with complete block of contrast agent?', 'is_impossible': False, 'id': '7896

In [9]:
from datasets import load_dataset
s = load_dataset('Saptarshi7/RadQA', field='data')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/803 [00:00<?, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [3]:
s = load_dataset('squad')

Found cached dataset squad (/home/saptarshi/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
s

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})